In [0]:
%matplotlib inline

In [0]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import itertools
import datetime as dt
import pickle
import os

In [58]:
!pip install dgl

In [0]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F

In [0]:
gcn_msg = fn.copy_src(src='h', out='m')
gcn_reduce = fn.sum(msg='m', out='h')

In [0]:
class GCNLayer(nn.Module):
  def __init__(self, in_feats, out_feats):
    super(GCNLayer, self).__init__()
    self.linear = nn.Linear(in_feats, out_feats)

  def forward(self, g, feature):
    with g.local_scope():
      g.ndata['h'] = feature
      g.update_all(gcn_msg, gcn_reduce)
      h = g.ndata['h']
      return self.linear(h)

In [0]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.layer1 = GCNLayer(20, 100)
    self.layer2 = GCNLayer(100, 50)
    self.layer3 = GCNLayer(50, 10)
    self.layer4 = GCNLayer(10, 2)

  def forward(self, g, features):
    x = F.relu(self.layer1(g, features))
    x = F.relu(self.layer2(g, x))
    x = F.relu(self.layer3(g, x))
    x = F.sigmoid(self.layer4(g, x))
    return x

In [0]:
def load_data(file):
  with open(f'model_ready_data/{file}', 'rb') as f:
    C, features, targets = pickle.load(f)
  features = th.FloatTensor(features)
  targets = th.FloatTensor(targets)
  g = dgl.DGLGraph()
  g.from_networkx(C)
  return g, features, targets

In [0]:
def evaluate(model, g, features, targets):
  model.eval()
  with th.no_grad():
    

In [92]:
net = Net()
print(net)

Net(
  (layer1): GCNLayer(
    (linear): Linear(in_features=20, out_features=100, bias=True)
  )
  (layer2): GCNLayer(
    (linear): Linear(in_features=100, out_features=50, bias=True)
  )
  (layer3): GCNLayer(
    (linear): Linear(in_features=50, out_features=10, bias=True)
  )
  (layer4): GCNLayer(
    (linear): Linear(in_features=10, out_features=2, bias=True)
  )
)


In [0]:
optimizer = th.optim.Adam(net.parameters(), lr=1e-2)
n_epochs = 100
for epoch in range(n_epochs):
  loss_track = []
  for file in os.listdir('model_ready_data'):
    g, features, targets = load_data(file)
    
    net.train()
    pred = net(g, features)
    loss = F.mse_loss(pred, targets)
    loss_track.append(loss)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  
  print(f'Epoch: {epoch}, Average Loss: {sum(loss_track)/len(loss_track)}')

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 0, Average Loss: 0.009138769470155239
Epoch: 1, Average Loss: 0.004173393361270428
Epoch: 2, Average Loss: 0.004173393361270428
Epoch: 3, Average Loss: 0.004173393361270428
Epoch: 4, Average Loss: 0.004173393361270428
Epoch: 5, Average Loss: 0.004173393361270428
Epoch: 6, Average Loss: 0.004173393361270428
Epoch: 7, Average Loss: 0.004173393361270428
Epoch: 8, Average Loss: 0.004173393361270428
Epoch: 9, Average Loss: 0.004173393361270428
Epoch: 10, Average Loss: 0.004173393361270428
Epoch: 11, Average Loss: 0.004173393361270428
Epoch: 12, Average Loss: 0.004173393361270428
Epoch: 13, Average Loss: 0.004173393361270428
Epoch: 14, Average Loss: 0.004173393361270428
Epoch: 15, Average Loss: 0.004173393361270428


KeyboardInterrupt: ignored

In [0]:
g, features, targets = load_data('2020-05-08.pkl')

In [89]:
net(g, features) - targets

tensor([[-2.6079e-02, -2.1781e-02],
        [ 1.1839e-03, -7.5676e-03],
        [-1.6131e-02, -1.1569e-02],
        [-2.9533e-02, -2.6887e-02],
        [-9.0157e-03, -1.0540e-02],
        [-4.6816e-03, -1.2026e-02],
        [-1.9071e-01, -1.0676e-01],
        [-5.4656e-02, -4.3768e-02],
        [ 5.1554e-02,  2.3462e-02],
        [ 1.6670e-01,  1.5868e-01],
        [ 4.0324e-03, -1.5991e-04],
        [-1.1631e-01, -7.0784e-02],
        [-9.5073e-02, -6.0534e-02],
        [-4.2670e-03, -9.7396e-03],
        [-1.9299e-01, -1.0412e-01],
        [-5.0320e-02, -4.5763e-02],
        [-1.8944e-02, -3.6913e-03],
        [-1.7853e-02, -1.3588e-02],
        [ 1.6928e-02,  1.3458e-02],
        [-9.1060e-02, -9.2047e-02],
        [ 7.7990e-02,  6.6358e-02],
        [ 1.0733e-01,  9.3269e-02],
        [ 3.3014e-02,  5.0348e-02],
        [ 1.7158e-02,  4.4675e-02],
        [ 2.3935e-02, -2.0684e-02],
        [ 9.1067e-02,  8.5943e-02],
        [-2.5187e-02, -2.2772e-02],
        [-2.7133e-02, -2.559

In [88]:
targets

tensor([[2.8404e-02, 1.4594e-02],
        [1.1410e-03, 3.8105e-04],
        [1.8456e-02, 4.3821e-03],
        [3.1858e-02, 1.9700e-02],
        [1.1341e-02, 3.3533e-03],
        [7.0065e-03, 4.8394e-03],
        [1.9303e-01, 9.9569e-02],
        [5.6981e-02, 3.6581e-02],
        [9.8094e-02, 1.0951e-01],
        [1.8495e-02, 8.1165e-03],
        [1.7854e-02, 1.1584e-02],
        [1.1864e-01, 6.3598e-02],
        [9.7398e-02, 5.3348e-02],
        [6.5919e-03, 2.5531e-03],
        [2.2324e-01, 1.2350e-01],
        [7.0053e-02, 5.5139e-02],
        [3.4675e-02, 9.2596e-03],
        [2.0178e-02, 6.4017e-03],
        [1.9031e-02, 1.1355e-02],
        [9.3385e-02, 8.4861e-02],
        [4.1585e-03, 2.4006e-03],
        [8.5955e-04, 2.6674e-04],
        [9.2265e-02, 5.9444e-02],
        [2.2800e-01, 1.7917e-01],
        [1.4039e-01, 1.6763e-01],
        [3.0532e-02, 2.0348e-02],
        [2.7512e-02, 1.5585e-02],
        [2.9458e-02, 1.8405e-02],
        [1.3862e-03, 6.0969e-04],
        [2.366